# Navier-Stokes (cavity flow)

In [2]:
#importamos los paquetes
from ngsolve import *
from ngsolve.webgui import *
from netgen.occ import *
import netgen.geom2d as geom2d

In [64]:
def rectangle_mesh(a,b,c,d,h,graficar=True):
    #funcion que genera una malla rectangular en el dominio [a,b]x[c,d]
    geo = geom2d.SplineGeometry()
    p1 = geo.AppendPoint (a,c)
    p2 = geo.AppendPoint (b,c)
    p3 = geo.AppendPoint (b,d)
    p4 = geo.AppendPoint (a,d)

    geo.Append (["line", p1, p2],bc = "bottom")
    geo.Append (["line", p2, p3],bc = "right")
    geo.Append (["line", p3, p4],bc = "top")
    geo.Append (["line", p4, p1],bc = "left")

    mesh = Mesh(geo.GenerateMesh (maxh=h))
    #mesh.SplitElements_Alfeld()
    #mesh.GetBoundaries()
    if graficar == True:
        Draw(mesh)
    return mesh

def solveStokes(nu,mesh,X,gfu,graficar=True):
    (u,p,lam), (v,q,mu) = X.TnT()
    a = BilinearForm(X)
    stokes = (nu*InnerProduct(grad(u),grad(v))+div(u)*q-div(v)*p-lam*q-mu*p)*dx
    a += stokes
    a.Assemble()

    f = LinearForm(X)
    f.Assemble()

    res = f.vec - a.mat*gfu.vec
    inv_stokes = a.mat.Inverse(X.FreeDofs())
    gfu.vec.data += inv_stokes * res
    velocity, pressure, param = gfu.components
    if graficar == True:
        sceneu = Draw(velocity,mesh,"u")
        scenep = Draw(pressure,mesh,"p")

    return gfu

def LinNS(nu,mesh,X,gfu,w):
    (u,p,lam), (v,q,mu) = X.TnT()
    a = BilinearForm(X)
    LinNS = (nu*InnerProduct(grad(u),grad(v))+div(u)*q-div(v)*p+InnerProduct(grad(u)*w,v)+InnerProduct(grad(w)*u,v)-InnerProduct(grad(w)*w,v)-lam*q-mu*p)*dx
    a += LinNS
    a.Assemble()

    f = LinearForm(X)
    f.Assemble()

    res = f.vec - a.mat*gfu.vec
    inv_LinNS = a.mat.Inverse(X.FreeDofs())
    gfu.vec.data += inv_LinNS * res

    return gfu

    

In [83]:
#defino la malla
mesh = rectangle_mesh(0,1,0,1,1/16,graficar=False)
#defino los espacios
V = VectorH1(mesh, order=2, dirichlet="bottom|right|top|left")
Q = H1(mesh, order=1)
N = NumberSpace(mesh)
X = V*Q*N

#Datos Dirichlet
gfu = GridFunction(X)
velocity = gfu.components[0]

g = CoefficientFunction((1,0))
velocity.Set(g, definedon=mesh.Boundaries("top"))
#Draw(velocity,mesh)
#resuelvo Stokes
#stokes_solution = solveStokes(0.1,mesh,X,gfu,graficar=False)

#w = stokes_solution.components[0]
w=velocity
NS = LinNS(0.1,mesh,X,gfu,w)
w1=NS.components[0]
#Draw(w1,mesh)
import numpy as np
max(np.array(w.vec)-np.array(w1.vec))


0.0